# Equity Risk Factors Models with Python
### Portfolio Optimization Part III
<br>__[Financionerioncios](https://financioneroncios.wordpress.com)__
<br>__[Orenji](https://www.orenj-i.com)__
<br>__[Dany Cajas](https://www.linkedin.com/in/dany-cajas/)__

<br>Hola gente, espero les guste este post que es de mis temas favoritos =D. No se olviden que para Python les recomiendo que usen __[winpython](https://winpython.github.io/)__ (solo windows, pero es portable)  o __[anaconda](https://www.anaconda.com/download/)__ (es multiplataforma, pero requiere de instalación). Al instalar cualquiera de estas distribuciones encontraran los ejecutables Jupyter Notebook (el formato del post, los archivos tienen extensión es .ipynb) y Spyder (IDE que recomiendo para trabajar porque permite visualizar las variables y su tipo, los archivos tienen extensión es .py)

En la página del blog dejaré los notebooks (se abren con jupyter notebook), los scripts de python (recomiendo que usen spyder para trabajar con estos archivos) y los archivos en excel con la data usada en los ejemplos.

Los temas que trataremos en este post son:
1. Modelo de Capital Asset Princing Model (CAPM).
   1. Formulación del Modelo.
   1. Estimación de los parámetros del modelo de media varianza.
2. Modelo de Factores de Riesgo.
   1. Formulación del Modelo.
   1. Estimación de los parámetros del modelo de media varianza.

## 1. Modelo CAPM

### 1. Formulación del Modelo

El modelo CAPM propuesto por Sharpe, busca explicar el retorno de los activos como una función del activo libre de riesgo, un coeficiente $\beta_a$ y la prima de riesgo de mercado.
<br>
<br>$$ r_a = r_f + \beta_a (r_m - r_f) + \epsilon $$
<br> 
Donde:
<br>$r_a:$ es el retorno del activo a.
<br>$r_f:$ es la tasa libre de riesgo.
<br>$r_m:$ es el retorno del mercado.
<br>$r_m-r_f:$ es la prima de riesgo del mercado.
<br>$\epsilon:$ es el error de estimación del modelo.

El termino $\beta_a$ es calculado mediante una regresión entre el rendimiento del activo y el rendimiento de un indice de mercado (usualmente el S&P500), por lo que la formula para calcular el $\beta_a$ es la siguiente:
<br>
<br>$$ \beta_a= \frac{cov(r_a,r_m)}{\sigma_m^2} = \rho_{a,m}\frac{\sigma_a}{\sigma_m}$$
<br> 
Donde:
<br>$cov(r_a,r_m):$ es la covarianza entre los retornos del activo y los retornos del mercado.
<br>$\rho_{a,m}:$ es la correlación entre los retornos del activo y los retornos del mercado.
<br>$\sigma_m:$ es la desviación estándar de los retornos del mercado o la volatilidad del mercado.
<br>$\sigma_a:$ es la desviación estándar de los retornos del activo o la volatilidad del activo.

### 2. Estimación de los parámetros del modelo de media varianza.

A partir del modelo CAPM podemos estimar los inputs del modelo de media varianza o modelo de Markowitz.

El vector de medias de medias y la matriz de covarianzas pueden estimarse con las siguientes formulas:
<br>

$$
\begin{equation*}
\begin{gathered}
E(r) = r_f + B E(r_m - r_f) \\
\\
\Sigma = B \sigma_{r_m - r_f} B^T + \Sigma_{\epsilon} \\
\\
E(r) = \begin{bmatrix} E(r_{1}) \\ E(r_{2}) \\ \vdots \\ E(r_{N}) \end{bmatrix} \; \;  , \; \;
\alpha =  \begin{bmatrix} \alpha_{1} \\ \alpha_{2} \\ \vdots \\ \alpha_{N} \end{bmatrix} \; \;  , \; \;
B = \begin{bmatrix} \beta_{1,1} \\ \beta_{2,1} \\ \vdots \\ \beta_{N,1} \end{bmatrix} \; \;  , \; \;
\epsilon = \begin{bmatrix} \epsilon_{1} \\ \epsilon_{2} \\ \vdots \\ \epsilon_{N} \end{bmatrix}
\end{gathered}
\end{equation*}
$$

Donde:
<br>$E(r):$ es el vector de retornos esperados de los activos.
<br>$B:$ es el vector de betas de los activos.
<br>$\Sigma:$ es la matriz de covarianzas de los activos.
<br>$\sigma_{r_m - r_f}:$ es la varianza de la prima de mercado.
<br>$\Sigma_{\epsilon}:$ es una matriz diagonal de las varianzas de los errores de cada activo.


In [2]:
########################################################################
# Importando la Data
########################################################################

import pandas as pd
import numpy as np

sp500 = pd.read_excel('SP500.xlsx', index_col=0, header=0)
sp500_ret = sp500.pct_change().dropna()

precios = pd.read_excel('Prices.xlsx', index_col=0, header=0)
precios = precios.sample(n = 10, random_state = 1, axis = 1)
retornos = precios.pct_change().dropna()

assets = precios.columns.tolist()


In [3]:
########################################################################
# Calculando los Betas
########################################################################
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Usando statsmodels
x = sp500_ret.values
x = sm.add_constant(x)

Beta_1 = []
for i in assets:
    y_i = retornos[i].values
    mod = sm.OLS(y_i, x)
    res = mod.fit()
    Beta_1.append(res.params[1])

Beta_1 = np.matrix(Beta_1)

# Usando scikit learn
y = retornos.values
x = sp500_ret.values

print(x)
print(y)
reg = LinearRegression().fit(x, y)
Beta_2 = np.matrix(reg.coef_).T

# Combinando ambos betas para mostrarlos
B1 = pd.DataFrame(Beta_1.T, index = assets, columns = ['Beta_1'])
B2 = pd.DataFrame(Beta_2.T, index = assets, columns = ['Beta_2'])

display(pd.concat([B1, B2], axis = 1))


[[-2.01080833e-02]
 [ 9.67198283e-02]
 [-3.07957556e-02]
 [-2.19150505e-02]
 [ 2.39335496e-02]
 [-1.63412762e-02]
 [ 6.06991052e-02]
 [-5.34829747e-02]
 [-4.94948587e-03]
 [-8.00686100e-02]
 [ 4.05344553e-03]
 [ 3.46365771e-02]
 [-9.22907358e-02]
 [-6.42047105e-02]
 [ 7.68143618e-02]
 [ 5.09018712e-03]
 [-2.50353891e-02]
 [-1.07401501e-02]
 [-6.41084066e-02]
 [-8.17234060e-02]
 [ 1.80991114e-02]
 [ 7.51759043e-02]
 [ 7.57383419e-03]
 [-1.55738326e-02]
 [-2.07662113e-02]
 [ 3.67388936e-02]
 [-6.14176262e-02]
 [-9.08148108e-03]
 [-7.24553544e-02]
 [-7.90042750e-02]
 [ 4.88143308e-03]
 [-1.10024318e-01]
 [ 8.64488010e-02]
 [ 5.70696209e-02]
 [-6.03325727e-02]
 [-2.74146926e-02]
 [-1.70036085e-02]
 [ 8.35756857e-03]
 [ 8.10441069e-02]
 [ 5.08987097e-02]
 [ 1.13222145e-02]
 [ 1.62237024e-02]
 [ 1.78732034e-02]
 [-1.19443645e-02]
 [ 5.49614864e-02]
 [ 7.12850385e-03]
 [ 5.07655410e-02]
 [ 1.72763870e-02]
 [ 1.22089733e-02]
 [-1.63589192e-02]
 [-1.67907519e-02]
 [ 1.20834502e-02]
 [ 1.7988998

Beta_1    Beta_2
NTRS  0.995139  0.995139
O     0.431426  0.431426
SBUX  0.902596  0.902596
SWK   1.188754  1.188754
COF   1.552629  1.552629
M     1.313719  1.313719
ECL   0.650029  0.650029
EXC   0.338666  0.338666
HRB   0.455073  0.455073
ALXN  1.088635  1.088635

In [3]:
########################################################################
# Calculando el vector de medias y la matriz de covarianzas a partir
# del modelo CAPM
########################################################################

Sigma_rm = np.matrix(np.var(sp500_ret, ddof = 1))

# Usando statsmodels
x = sp500_ret.values
x = sm.add_constant(x)

mu_F_1 = []
Sigma_e_1 = []
for i in assets:
    y_i = retornos[i].values
    mod = sm.OLS(y_i, x)
    res = mod.fit()
    error = np.matrix(y_i) - np.matrix(res.predict(x))
    mu_F_1.append(np.mean(res.predict(x), axis = 0))
    Sigma_e_1.append(np.asscalar(np.var(error.T, ddof = 1, axis = 0)))

mu_F_1 = np.matrix(mu_F_1)
Sigma_e_1 = np.diag(Sigma_e_1)
Sigma_F_1 = Beta_2.T * Sigma_rm *  Beta_2 + Sigma_e_1

display(pd.DataFrame(mu_F_1, columns = assets))
display(pd.DataFrame(Sigma_F_1, index = assets, columns = assets))

NTRS         O      SBUX       SWK       COF         M       ECL  \
0  0.004512  0.009679  0.016238  0.010317  0.009858  0.008295  0.011343   

        EXC       HRB      ALXN  
0  0.005316  0.007505  0.021858

NTRS         O      SBUX       SWK       COF         M       ECL  \
NTRS  0.004613  0.000739  0.001546  0.002036  0.002659  0.002250  0.001113   
O     0.000739  0.003413  0.000670  0.000883  0.001153  0.000975  0.000483   
SBUX  0.001546  0.000670  0.007343  0.001847  0.002412  0.002041  0.001010   
SWK   0.002036  0.000883  0.001847  0.006047  0.003177  0.002688  0.001330   
COF   0.002659  0.001153  0.002412  0.003177  0.012339  0.003510  0.001737   
M     0.002250  0.000975  0.002041  0.002688  0.003510  0.012216  0.001470   
ECL   0.001113  0.000483  0.001010  0.001330  0.001737  0.001470  0.003057   
EXC   0.000580  0.000251  0.000526  0.000693  0.000905  0.000766  0.000379   
HRB   0.000779  0.000338  0.000707  0.000931  0.001216  0.001029  0.000509   
ALXN  0.001864  0.000808  0.001691  0.002227  0.002909  0.002461  0.001218   

           EXC       HRB      ALXN  
NTRS  0.000580  0.000779  0.001864  
O     0.000251  0.000338  0.000808  
SBUX  0.000526  0.000707  0.001691  
SWK   0.000693  0.000931  0.002227  
COF   0.000905  0.001216  0.002909  
M     0.000766  0.001029  0.002461  
ECL   0.000379  0.000509  0.001218  
EXC   0.003506  0.000265  0.000635  
HRB   0.000265  0.006797  0.000853  
ALXN  0.000635  0.000853  0.025560

In [4]:
# Usando scikit learn
x = sp500_ret.values

error_2 = y - reg.predict(x)

mu_F_2 = np.matrix(np.mean(reg.predict(x), axis = 0))
Sigma_e_2 = np.diag(np.var(error_2, ddof = 1, axis = 0))
Sigma_F_2 = Beta_2.T * Sigma_rm *  Beta_2 + Sigma_e_2

display(pd.DataFrame(mu_F_2, columns = assets))
display(pd.DataFrame(Sigma_F_2, index = assets, columns = assets))

#display(pd.DataFrame(np.cov(y.T), index = assets, columns = assets))

NTRS         O      SBUX       SWK       COF         M       ECL  \
0  0.004512  0.009679  0.016238  0.010317  0.009858  0.008295  0.011343   

        EXC       HRB      ALXN  
0  0.005316  0.007505  0.021858

NTRS         O      SBUX       SWK       COF         M       ECL  \
NTRS  0.004613  0.000739  0.001546  0.002036  0.002659  0.002250  0.001113   
O     0.000739  0.003413  0.000670  0.000883  0.001153  0.000975  0.000483   
SBUX  0.001546  0.000670  0.007343  0.001847  0.002412  0.002041  0.001010   
SWK   0.002036  0.000883  0.001847  0.006047  0.003177  0.002688  0.001330   
COF   0.002659  0.001153  0.002412  0.003177  0.012339  0.003510  0.001737   
M     0.002250  0.000975  0.002041  0.002688  0.003510  0.012216  0.001470   
ECL   0.001113  0.000483  0.001010  0.001330  0.001737  0.001470  0.003057   
EXC   0.000580  0.000251  0.000526  0.000693  0.000905  0.000766  0.000379   
HRB   0.000779  0.000338  0.000707  0.000931  0.001216  0.001029  0.000509   
ALXN  0.001864  0.000808  0.001691  0.002227  0.002909  0.002461  0.001218   

           EXC       HRB      ALXN  
NTRS  0.000580  0.000779  0.001864  
O     0.000251  0.000338  0.000808  
SBUX  0.000526  0.000707  0.001691  
SWK   0.000693  0.000931  0.002227  
COF   0.000905  0.001216  0.002909  
M     0.000766  0.001029  0.002461  
ECL   0.000379  0.000509  0.001218  
EXC   0.003506  0.000265  0.000635  
HRB   0.000265  0.006797  0.000853  
ALXN  0.000635  0.000853  0.025560

Ahora vamos a comparar el resultado de optimizar un portafolio usando las estimaciones del CAPM versus el modelo historico.

In [5]:
########################################################################
# Calculando los portafolios optimos con las estimaciones historicas 
# y las estimaciones del CAPM
########################################################################

import cvxpy as cv

# Definiendo la funcion de optimización
def Max_Sharpe(mu, Sigma):
    w = cv.Variable((mu.shape[1], 1))
    k = cv.Variable(1)
    rf = cv.Parameter(nonneg=True)
    rf.value=0
    #definiendo el problema, funcion objetivo y reestricciones    
    constraints = [(mu - rf)*w == 1,
                   w >= 1e-10,
                   k >= 1e-10,
                   cv.sum(w) == k]
    prob = cv.Problem(cv.Minimize(cv.quad_form(w, Sigma)), constraints)
    #resolviendo el problema
    prob.solve(solver=cv.ECOS)

    return np.matrix(w.value/k.value)

# Definiendo los inputs del modelo historico
mu = np.matrix(np.mean(retornos.values, axis = 0))
Sigma = np.cov(retornos.values.T)

w1 = Max_Sharpe(mu, Sigma)
w2 = Max_Sharpe(mu_F_2, Sigma_F_2)

# Combinando los portafolios para mostrarlos
W1 = pd.DataFrame(w1, index = assets, columns = ['Historico'])
W2 = pd.DataFrame(w2, index = assets, columns = ['CAPM'])

display(pd.concat([W1, W2], axis = 1).applymap("{0:.2%}".format))

Historico    CAPM
NTRS     0.00%   0.00%
O       21.91%  24.97%
SBUX    15.77%  18.53%
SWK      0.11%   0.80%
COF      0.00%   0.00%
M        0.00%   0.00%
ECL     37.73%  32.09%
EXC      8.72%   9.47%
HRB      5.15%   7.26%
ALXN    10.62%   6.88%

### 2. Risk Factors

Como una alternativa al Capital Asset Pricing Model (CAPM), Stephen Ross derivó un modelo de precios de activos basado en argumentos de arbitraje denominado Arbitrage Price Theory (APT). La APT establece que el rendimiento esperado de un activo está linealmente relacionado con varios factores de riesgo, a diferencia del CAPM que sugiere que solo esta relacionado con el rendimiento de mercado. Sin embargo, la APT no especifica cuáles son estos factores de riesgo, esta es una ventaja ya que permite modelar el riesgo de forma mas libre que el CAPM, además que este enfoque permite eliminar el riesgo no sistemático para que un inversionista solo sea compensado por aceptar los factores de riesgo sistemáticos. 

### A. Formulación
Un modelo de múltiples factores de riesgo comienza suponiendo que tenemos $N$ activos y que el retorno de cada activo puede expresarse de la forma:

$$
\begin{gathered}
R_{i} = \alpha_{i} + \sum^{K}_{i = 1}\beta_{i,k}F_{k} + \epsilon_{i}
\end{gathered}
$$

Además se asume:

$$
\begin{gathered}
E(\epsilon_{i})=0 \\
\\
E(\epsilon_{i}\epsilon_{j}) = E(\epsilon_{i}F_{j})= 0 \\
\\
E(\epsilon_{i}\epsilon_{i}) = \sigma_{i}^{2} \; ; \; \forall \; i \neq j \in N
\end{gathered}
$$

Aquí $F_{k} \; \forall \; k = 1,2,\cdots,K$ son $K$ factores de riesgo comunes a todos los activos, $\beta_{ik}$ es la sensibilidad del activo $i$ al factor $k$, y $\epsilon_{i}$ es el retorno no sistemático (idiosincrásico) para el activo $i$. Podemos escribir la relación anterior en forma matricial para todos los activos de la siguientes forma:

$$
\begin{equation*}
\begin{gathered}
R = \alpha + B F + \epsilon \\
\\
R =  \begin{bmatrix} R_{1} \\ R_{2} \\ \vdots \\ R_{N} \end{bmatrix} \; \;  , \; \;
\alpha =  \begin{bmatrix} \alpha_{1} \\ \alpha_{2} \\ \vdots \\ \alpha_{N} \end{bmatrix} \; \;  , \; \;
B = \begin{bmatrix} \beta_{1,1} & \beta_{1,2} & \cdots &\beta_{1,k}\\ \beta_{2,1} & \beta_{2,2} & \cdots &\beta_{2,k} \\ \vdots & \vdots & \ddots & \vdots \\ \beta_{N,1} & \beta_{N,2} & \cdots &\beta_{N,k} \end{bmatrix} \; \;  , \; \;
F = \begin{bmatrix} F_{1} \\ F_{2} \\ \vdots \\ F_{k} \end{bmatrix} \; \; , \; \; 
\epsilon = \begin{bmatrix} \epsilon_{1} \\ \epsilon_{2} \\ \vdots \\ \epsilon_{N} \end{bmatrix}
\end{gathered}
\end{equation*}
$$

Donde a la matriz $B$ se le conoce como matriz de mapeo de factores o loadings matrix. Si tomamos valor esperado a la ecuación anterior, obtenemos el vector de retornos esperados de los activos $\mu$, expresado en términos de los retornos esperados de los factores de riesgo:

$$
\begin{gathered}
\mu = \alpha +BE(F)
\end{gathered}
$$

Para calcular la matriz de covarianzas $\Sigma_{f}$ siguiendo usando el modelo de factores, utilizamos la siguiente ecuación:

$$
\begin{gathered}
\Sigma_{f} = B \Sigma_{F} B^{T} + \Sigma_{\epsilon}
\end{gathered}
$$

Donde $\Sigma_{F}$ es la matriz de covarianzas de los factores de riesgo y $\Sigma_{\epsilon}$ es la matriz de covarianzas de los retornos no sistemáticos $\epsilon$, la cual por los supuestos iniciales es una matriz diagonal, donde cada elemento de la diagonal es $\sigma_{i}^{2}$. En la practica se puede aproximar la matriz de covarianzas $\Sigma_{f}$ con la ecuación $ \Sigma_{f} \approx B \Sigma_{F} B^{T}$, en caso no se tengan valores observados de los retornos de los activos para estimar $\epsilon_{i}$. Finalmente este método nos permite estimar parámetros de entrada más robustos que luego se utilizan en el modelo de media varianza.

Un problema de este método es que se puede generar una matriz de covarianzas $\Sigma_{f}$ que no sea definida positiva, lo que hace imposible la solución del problema de optimización; en este caso se puede optar por dos alternativas: hallar la matriz definida positiva mas cercana a $\Sigma_{f}$ o utilizar la matriz de covarianzas original de los activos $\Sigma$. 

### B. Modelo de Fama French

Un modelo de factores de riesgos muy conocido es el modelo de tres factores de Fama French, el cual usaremos en este post y tiene la siguiente forma:

$$
\begin{gathered}
r_{i} - r_{f} = \beta_{0} + \beta_{1} (r_{m} - r_{f}) + \beta_{2}SMB + \beta_{3}HML + \epsilon
\end{gathered}
$$

<br>Donde:
<br>$r - r_{f}:$ es el retorno en exceso del activo.
<br>$r_{m} - r_{f}:$ es la prima de riesgo de mercado.
<br>$SMB:$ es el factor tamaño o small minus big market cap, este factor nos dice que las empresas de menor <br>capitalización de mercado tienen un rendimiento superior que las empresas de mayor capitalización.
<br>$HML:$ es el factor valor o high minus low price to book, este factor nos dice que las empresas con un alto ratio price to book tienen un rendimiento superior que las empresas de bajo ratio price to book.

La ecuación anterior puede ser expresada en la forma matricial del modelo de factores de la sección anterior para un número de activos $N$, usando los siguientes arreglos:

$$
\begin{equation*}
\begin{gathered}
R = \alpha + B F + \epsilon \\
\\
R =  \begin{bmatrix} r_{1} \\ r_{2} \\ \vdots \\ r_{N} \end{bmatrix} \; \;  , \; \;
\alpha = \begin{bmatrix} r_{f} + \beta_{1,0} \\  r_{f} + \beta_{2,0} \\ \vdots \\  r_{f} + \beta_{N,0} \end{bmatrix} \; \;  , \; \;
B = \begin{bmatrix} \beta_{1,1} & \beta_{1,2} & \beta_{1,3}\\ \beta_{2,1} & \beta_{2,2} &\beta_{2,3} \\ \vdots & \vdots & \vdots \\ \beta_{N,1} & \beta_{N,2} &\beta_{N,3} \end{bmatrix} \; \;  , \; \;
F = \begin{bmatrix} r_{m} - r_{f} \\ SMB \\ HML \end{bmatrix}
\end{gathered}
\end{equation*}
$$


In [6]:
########################################################################
# Calculando el vector de medias y la matriz de covarianzas a partir
# del modelo CAPM
########################################################################

# Cargando la informacion de los factores
Factores = pd.read_excel('FamaFrench.xlsx', index_col=0, header=0)/100

# Calculando los betas del modelo
x = Factores[['Mkt-RF', 'SMB', 'HML']].values
reg = LinearRegression().fit(x, y)
B = np.matrix(reg.coef_).T

# Calculando los inputs del modelo
Sigma_F = np.cov(x.T)
error = y - reg.predict(x)
Sigma_e = np.diag(np.var(error, ddof = 1, axis = 0))

mu_fama = np.matrix(np.mean(reg.predict(x), axis = 0))
Sigma_fama = B.T * Sigma_F *  B + Sigma_e_2

display(pd.DataFrame(mu_fama, columns = assets))
display(pd.DataFrame(Sigma_fama, index = assets, columns = assets))


NTRS         O      SBUX       SWK       COF         M       ECL  \
0  0.004512  0.009679  0.016238  0.010317  0.009858  0.008295  0.011343   

        EXC       HRB      ALXN  
0  0.005316  0.007505  0.021858

NTRS         O      SBUX       SWK       COF         M       ECL  \
NTRS  0.004814  0.000780  0.001544  0.002230  0.003265  0.002501  0.001253   
O     0.000780  0.003525  0.000783  0.001016  0.001519  0.001225  0.000424   
SBUX  0.001544  0.000783  0.007595  0.001948  0.002587  0.002343  0.000803   
SWK   0.002230  0.001016  0.001948  0.006318  0.003980  0.003111  0.001384   
COF   0.003265  0.001519  0.002587  0.003980  0.014527  0.004538  0.002143   
M     0.002501  0.001225  0.002343  0.003111  0.004538  0.012897  0.001464   
ECL   0.001253  0.000424  0.000803  0.001384  0.002143  0.001464  0.003268   
EXC   0.000709  0.000249  0.000411  0.000789  0.001307  0.000831  0.000549   
HRB   0.000874  0.000454  0.000874  0.001111  0.001597  0.001327  0.000482   
ALXN  0.001157  0.000778  0.002507  0.001654  0.000740  0.002335 -0.000018   

           EXC       HRB      ALXN  
NTRS  0.000709  0.000874  0.001157  
O     0.000249  0.000454  0.000778  
SBUX  0.000411  0.000874  0.002507  
SWK   0.000789  0.001111  0.001654  
COF   0.001307  0.001597  0.000740  
M     0.000831  0.001327  0.002335  
ECL   0.000549  0.000482 -0.000018  
EXC   0.003645  0.000270 -0.000301  
HRB   0.000270  0.006930  0.001004  
ALXN -0.000301  0.001004  0.031738

In [7]:
########################################################################
# Calculando los portafolios optimos con las estimaciones del modelo de 
# factores de riesgo de Fama French
########################################################################

w3 = Max_Sharpe(mu_fama, Sigma_fama)

# Combinando los portafolios para mostrarlos
W3 = pd.DataFrame(w3, index = assets, columns = ['Factores'])

display(pd.concat([W1, W2, W3], axis = 1).applymap("{0:.2%}".format))

Historico    CAPM Factores
NTRS     0.00%   0.00%    0.00%
O       21.91%  24.97%   24.27%
SBUX    15.77%  18.53%   17.91%
SWK      0.11%   0.80%    0.00%
COF      0.00%   0.00%    0.00%
M        0.00%   0.00%    0.00%
ECL     37.73%  32.09%   34.67%
EXC      8.72%   9.47%   10.02%
HRB      5.15%   7.26%    6.34%
ALXN    10.62%   6.88%    6.79%